In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.models as models
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import confusion_matrix
from torchvision import datasets, transforms
import numpy as np 
import timm
import torch.optim as optim
import random
from torch.optim import Adam

In [3]:

# Set seed for PyTorch
torch.manual_seed(42)

# Set seed for NumPy
np.random.seed(42)

# Set seed for Python random module
random.seed(42)


# Define paths to your data folders
train_data_dir = '/u/45/muhammu2/data/Desktop/spoofing/train/'
val_data_dir = '/u/45/muhammu2/data/Desktop/spoofing/validation/'
test_data_dir = '/u/45/muhammu2/data/Desktop/spoofing/test/'

# Example transformations (customize based on your requirements)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load your training, validation, and test datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_data_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)

# Print the number of images in each dataset
print(f"Number of images in the training set: {len(train_dataset)}")
print(f"Number of images in the validation set: {len(val_dataset)}")
print(f"Number of images in the test set: {len(test_dataset)}")

# Create DataLoader for training, validation, and test sets
batch_size = 4  # Adjust according to your needs
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


Number of images in the training set: 720
Number of images in the validation set: 1000
Number of images in the test set: 1800


In [4]:
# Define your SimpleDenseNet model (as defined previously)
class SimpleDenseNet(nn.Module):
    def __init__(self, num_classes):
        super(SimpleDenseNet, self).__init__()
        self.model = timm.create_model('densenet201', pretrained=True)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Instantiate the custom DenseNet model
num_classes = 2  # Binary classification
model = SimpleDenseNet(num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#optimizer = Adam(model.parameters(), lr=0.001)

In [5]:

num_epochs =  20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize EER and threshold variables
eer = None
threshold = None

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()

    all_labels = []
    all_scores = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            scores = torch.nn.functional.softmax(outputs, dim=1)[:, 1]

            all_labels.extend(labels.cpu().numpy())
            all_scores.extend(scores.cpu().numpy())

    # Calculate EER once after training on the validation set
    fpr, tpr, thresholds = roc_curve(all_labels, all_scores, pos_label=1)

    # Check for NaN values in the arrays
    if any(np.isnan(fpr)) or any(np.isnan(tpr)) or any(np.isnan(thresholds)):
        print("Error: NaN values encountered in fpr, tpr, or thresholds during EER calculation.")
        # Handle this case accordingly
    else:
        eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
        threshold = thresholds[np.nanargmin(np.abs(fpr - eer))]

        print(f"Epoch {epoch + 1}/{num_epochs}, Validation EER: {eer * 100:.2f}%")
        print(f"Validation EER Threshold: {threshold:.4f}")

Epoch 1/5, Validation EER: 4.76%
Validation EER Threshold: 0.2489
Epoch 2/5, Validation EER: 3.33%
Validation EER Threshold: 0.4218
Epoch 3/5, Validation EER: 1.90%
Validation EER Threshold: 0.2696
Epoch 4/5, Validation EER: 2.41%
Validation EER Threshold: 0.3995
Epoch 5/5, Validation EER: 3.16%
Validation EER Threshold: 0.0514


In [6]:

if eer is not None and threshold is not None:
    model.eval()

    genuine_scores = []
    impostor_scores = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            scores_test = torch.nn.functional.softmax(outputs, dim=1)[:, 1].cpu().detach().numpy()

            true_labels.extend(labels.cpu().numpy())
            impostor_scores.extend(scores_test[labels.cpu() == 0])  # Impostor scores
            genuine_scores.extend(scores_test[labels.cpu() == 1])   # Genuine scores

    # Apply threshold to impostor scores to get binary predictions
    binary_predictions = (np.array(impostor_scores) > threshold).astype(int)

    # Calculate FAR and FRR
    false_acceptance = sum(binary_predictions) / len(binary_predictions)
    false_rejection = sum(genuine_score <= threshold for genuine_score in genuine_scores) / len(genuine_scores)

    # Calculate HTER
    hter = 0.5 * (false_acceptance + false_rejection)

    # Calculate AUC
    auc = roc_auc_score(np.array(true_labels), np.concatenate([impostor_scores, genuine_scores]))

    print(f"Test HTER: {hter * 100:.2f}%")
    print(f"Test AUC: {auc:.4f}")
else:
    print("Error: EER and threshold not calculated during validation.")
    # Calculate the AUC on the testing set
    from sklearn.metrics import roc_auc_score
    auc_score = roc_auc_score(labels, scores_test.cpu())
    print(f"AUC on the testing set: {auc_score*100:.2f}")


Test HTER: 42.81%
Test AUC: 0.8078


In [28]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import numpy as np

# Assuming you have defined the model, optimizer, and criterion earlier

num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize EER and threshold variables
eer = None
threshold = None

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        # Logit mixing
        alpha = 10.5  # You can adjust the mixing ratio as needed
        mixed_outputs = alpha * outputs + (1 - alpha) * torch.flip(outputs, dims=[1])

        loss = criterion(mixed_outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()

    all_labels = []
    all_scores = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)

            # Logit mixing during validation
            mixed_outputs = alpha * outputs + (1 - alpha) * torch.flip(outputs, dims=[1])

            scores = F.softmax(mixed_outputs, dim=1)[:, 1]

            all_labels.extend(labels.cpu().numpy())
            all_scores.extend(scores.cpu().numpy())

    # Calculate EER once after training on the validation set
    fpr, tpr, thresholds = roc_curve(all_labels, all_scores, pos_label=1)

    # Check for NaN values in the arrays
    if any(np.isnan(fpr)) or any(np.isnan(tpr)) or any(np.isnan(thresholds)):
        print("Error: NaN values encountered in fpr, tpr, or thresholds during EER calculation.")
        # Handle this case accordingly
    else:
        eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
        threshold = thresholds[np.nanargmin(np.abs(fpr - eer))]

        print(f"Epoch {epoch + 1}/{num_epochs}, Validation EER: {eer * 100:.2f}%")
        print(f"Validation EER Threshold: {threshold:.4f}")


Epoch 1/5, Validation EER: 50.00%
Validation EER Threshold: inf
Epoch 2/5, Validation EER: 50.00%
Validation EER Threshold: inf
Epoch 3/5, Validation EER: 50.00%
Validation EER Threshold: inf
Epoch 4/5, Validation EER: 50.00%
Validation EER Threshold: inf
Epoch 5/5, Validation EER: 50.00%
Validation EER Threshold: inf


In [27]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score

if eer is not None and threshold is not None:
    model.eval()

    genuine_scores = []
    impostor_scores = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits = model(inputs)
            
            # Logits mixing with a ratio (alpha)
            alpha = 1.5  # You can adjust the mixing ratio as needed
            mixed_logits = alpha * logits + (1 - alpha) * torch.flip(logits, dims=[1])
            
            scores_test = torch.nn.functional.softmax(mixed_logits, dim=1)[:, 1].cpu().detach().numpy()

            true_labels.extend(labels.cpu().numpy())
            impostor_scores.extend(scores_test[labels.cpu() == 0])  # Impostor scores
            genuine_scores.extend(scores_test[labels.cpu() == 1])   # Genuine scores

    # Apply threshold to impostor scores to get binary predictions
    binary_predictions = (np.array(impostor_scores) > threshold).astype(int)

    # Calculate FAR and FRR
    false_acceptance = sum(binary_predictions) / len(binary_predictions)
    false_rejection = sum(genuine_score <= threshold for genuine_score in genuine_scores) / len(genuine_scores)

    # Calculate HTER
    hter = 0.5 * (false_acceptance + false_rejection)

    # Calculate AUC
    auc = roc_auc_score(np.array(true_labels), np.concatenate([impostor_scores, genuine_scores]))

    print(f"Test HTER: {hter * 100:.2f}%")
    print(f"Test AUC: {auc:.4f}")
else:
    print("Error: EER and threshold not calculated during validation.")


Test HTER: 40.35%
Test AUC: 0.8212
